## Import libaries

In [1]:
import os
import sys
assert os.path.exists('../src'), f"[ERROR] The path src not detected in the parent directory '{os.getcwd()}'."

if os.getcwd().endswith('/notebooks'):
    os.chdir('..')
    sys.path.append('./src')

print(f'[INFO] Current Directory: "{os.getcwd()}".')

[INFO] Current Directory: "/home/iceking/Desktop/22SS-TUM Lecture Docs/3. Praktikum (IN2106, IN4249)/Repo/topic-modeling-advancements".


In [11]:
%load_ext autoreload
%autoreload 2


from src import preprocessor
from src.utils import load_documents
import nltk
import json
import ast
from time import time
import shutil

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
combinations = []
with open('combinations.txt') as comb_file:
    for item in comb_file.readlines():
        #item = item.replace("'",'"')
        x= ast.literal_eval(item)
        combinations.append(x)

In [ ]:
from src.bulk_runner import OUTPUT_FOLDER
from src.bulk_runner import main_runner
if os.path.exists(OUTPUT_FOLDER):
    shutil.rmtree(OUTPUT_FOLDER)
dfs_output_topic_word = []
for i,configs in enumerate(combinations):
    print(f'[INFO] Configs #{i}/{len(combinations)}: {configs}')
    try:
        df_output_topic_word = main_runner(configs=configs)
        dfs_output_topic_word.append(df_output_topic_word)
    except Exception:
        print('[WARN] Current execution gave an error!')
        sleep(10)
        continue
pd.concat(dfs_output_topic_word).to_csv(f'./{OUTPUT_FOLDER}/merged.csv')

[INFO] Configs #0/1629: {'algorithm_args': {'embedding_model': 'all-distilroberta-v1', 'top_n_words': 5, 'algorithm': 'bertopic', 'n_gram_range_tuple': (1, 1), 'min_docs_per_topic': 15, 'num_topics': 4, 'cluster_model': 'hdbscan', 'hdbscan_args': {'min_cluster_size': 5, 'metric': 'euclidean', 'cluster_selection_method': 'eom', 'prediction_data': True}, 'umap_args': {'n_neighbors': 15, 'n_components': 3, 'min_dist': 0.0, 'metric': 'cosine', 'low_memory': False, 'random_state': 42}}, 'preprocessing_funcs': [], 'dataset': 'crisis_12'}
[WARN] [PREPROCESSOR] Preprocessing functions are empty or None, given:"[]", preprocessing is skipped.
[INFO] Running with 4 topics.
[INFO] The embedding model folder:"./pretrained_models/sentence-transformers_all-mpnet-base-v2" found, so no need to download.
[INFO] The embedding model folder:"./pretrained_models/sentence-transformers_all-distilroberta-v1" found, so no need to download.
[INFO] The embedding model folder:"./pretrained_models/sentence-transfor

Batches:   0%|          | 0/250 [00:00<?, ?it/s]

[INFO] Embeddings encoded successfully.
[INFO] Embedding Model created with SentenceTransformer.


2022-07-24 01:15:20,442 - BERTopic - Reduced dimensionality
2022-07-24 01:15:20,766 - BERTopic - Clustered reduced embeddings
2022-07-24 01:15:22,186 - BERTopic - Reduced number of topics from 257 to 5


[INFO] Creating Topic Similarity Matrix..


...

[0724/011536.567411:WARNING:sandbox_linux.cc(376)] InitializeSandbox() called with multiple threads in process gpu-process.
[0724/011536.575983:WARNING:bluez_dbus_manager.cc(247)] Floss manager not present, cannot set Floss enable/disable.


[INFO] Created Top Words Barchart Visualization successfully.
[INFO] Configs #1/1629: {'algorithm_args': {'embedding_model': 'paraphrase-multilingual-MiniLM-L12-v2', 'min_count': 10, 'algorithm': 'top2vec', 'num_topics': 2, 'doc2vec_speed': 'learn', 'umap_args': {'n_neighbors': 15, 'n_components': 3, 'metric': 'cosine', 'random_state': 42}, 'hdbscan_args': {'min_cluster_size': 15, 'metric': 'euclidean', 'cluster_selection_method': 'eom'}}, 'preprocessing_funcs': ['to_lowercase', 'standardize_accented_chars', 'remove_url', 'expand_contractions', 'expand_missing_delimiter', 'remove_mentions', 'remove_hashtags', 'remove_new_lines', 'remove_html_tags', 'keep_only_alphabet', 'remove_extra_spaces', 'remove_english_stop_words', 'lemmatize_noun', 'lemmatize_verb', 'lemmatize_adjective'], 'dataset': 'crisis_07'}
[INFO] [PREPROCESSOR] Available Preprocessing Functions in the Module:['to_lowercase', 'standardize_accented_chars', 'remove_url', 'expand_missing_delimiter', 'remove_mentions', 'remove

[0724/011536.997376:INFO:headless_shell.cc(660)] Written to file /home/iceking/Desktop/22SS-TUM Lecture Docs/3. Praktikum (IN2106, IN4249)/Repo/topic-modeling-advancements/output/1658617892_bertopic/vis_top_words_barchart.png.
2022-07-24 01:15:55,608 - top2vec - INFO - Pre-processing documents for training
2022-07-24 01:15:55,608 INFO:Pre-processing documents for training


[INFO] [PREPROCESSOR] Preprocessing completed in 18.542 seconds..
[INFO] Running with 2 topics.
[INFO] Top2Vec Parameters:
{
    "embedding_model": "paraphrase-multilingual-MiniLM-L12-v2",
    "min_count": 10,
    "algorithm": "top2vec",
    "num_topics": 2,
    "doc2vec_speed": "learn",
    "umap_args": {
        "n_neighbors": 15,
        "n_components": 3,
        "metric": "cosine",
        "random_state": 42
    },
    "hdbscan_args": {
        "min_cluster_size": 15,
        "metric": "euclidean",
        "cluster_selection_method": "eom"
    },
    "data_name": "crisis_07",
    "run_id": 1658618137
}
[INFO] The embedding model folder:"./pretrained_models/sentence-transformers_distiluse-base-multilingual-cased" found, so no need to download.
[INFO] The embedding model folder:"./pretrained_models/sentence-transformers_all-MiniLM-L6-v2" found, so no need to download.
[INFO] The embedding model folder:"./pretrained_models/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2" 

2022-07-24 01:15:55,963 - top2vec - INFO - Loading paraphrase-multilingual-MiniLM-L12-v2 model at ./pretrained_models/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2
2022-07-24 01:15:55,963 INFO:Loading paraphrase-multilingual-MiniLM-L12-v2 model at ./pretrained_models/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2
2022-07-24 01:15:57,580 - top2vec - INFO - Creating joint document/word embedding
2022-07-24 01:15:57,580 INFO:Creating joint document/word embedding
